In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import json
from haversine import haversine

pd.options.display.max_columns=999
os.getcwd()
import gc
# Get teenagers and mothers only

# Get all of the POI places

In [2]:
# requested of Walmart

In [3]:
df_brand_info=pd.read_csv("/home/jian/Docs/POI/places_20200406/part-00000-tid-3169497161568699755-0a71d798-e636-4e2f-949e-02e4669f577c-43-c000.csv.gz",
                          compression="gzip",dtype=str,sep="|")
df_brand_info_wmt=df_brand_info[df_brand_info['brand_name'].apply(lambda x: "walmart" in x.lower())]
list_brand_names=df_brand_info_wmt['brand_name'].unique().tolist()
del df_brand_info
del df_brand_info_wmt
gc.collect()

0

In [4]:
list_files_POI=glob.glob("/home/jian/Docs/POI/places_20200406/*")
list_files_POI=[x for x in list_files_POI if "poi" in os.path.basename(x).lower()]

In [5]:
list_brand_names

['walmart', 'walmart inc']

In [6]:
df_walmart_location=pd.DataFrame()
for file in list_files_POI:
    df=pd.read_csv(file,dtype=str,compression="gzip",sep=",")
    for brand_name in list_brand_names:
        df_b=df[df['brands'].apply(lambda x: brand_name in str(x).lower())]
        df_walmart_location=df_walmart_location.append(df_b)
print(df_walmart_location.shape,df_walmart_location['safegraph_place_id'].nunique())
print(df_walmart_location['brands'].unique().tolist())
print(df_walmart_location['location_name'].unique().tolist())

(4693, 22) 4693
['Walmart']
['Walmart', 'Walmart Supercenter', 'Wal Mart Supercenter', 'Alice Walmart Center', 'Plainview Walmart Center', 'Morris Super Walmart']


In [7]:
df_walmart_location=df_walmart_location[['safegraph_place_id','location_name','brands','street_address','city','region','postal_code','latitude','longitude']]
df_walmart_location['latitude']=df_walmart_location['latitude'].astype(float)
df_walmart_location['longitude']=df_walmart_location['longitude'].astype(float)
df_walmart_location.head(2)

,safegraph_place_id,location_name,brands,street_address,city,region,postal_code,latitude,longitude
921,sg:43f385d5026e4fffb55bbd42f0a91302,Walmart,Walmart,2400 Veterans Memorial Pkwy,Orange City,FL,32763,28.913804,-81.278884
1045,sg:4a8dc2497b1441ca8292e9ed302dd966,Walmart,Walmart,201 Gateway Blvd,Rock Springs,WY,82901,41.580524,-109.251274


In [8]:
df_walmart_location

,safegraph_place_id,location_name,brands,street_address,city,region,postal_code,latitude,longitude
921,sg:43f385d5026e4fffb55bbd42f0a91302,Walmart,Walmart,2400 Veterans Memorial Pkwy,Orange City,FL,32763,28.913804,-81.278884
1045,sg:4a8dc2497b1441ca8292e9ed302dd966,Walmart,Walmart,201 Gateway Blvd,Rock Springs,WY,82901,41.580524,-109.251274
2126,sg:9c63817bf6194d4b9e696d06444310ac,Walmart,Walmart,12000 Iron Bridge Rd,Chester,VA,23831,37.350503,-77.482775
2741,sg:cbe350b708b44a02b341ed66b386942f,Walmart,Walmart,4115 Calumet Ave,Manitowoc,WI,54220,44.075956,-87.696784
3303,sg:f86b790b3e98461a9493853531b5e46c,Walmart,Walmart,2310 E Serene Ave,Las Vegas,NV,89123,36.022987,-115.124299
...,...,...,...,...,...,...,...,...,...
532578,sg:2aeb24b60c1144abb891f0c5d77df2a1,Walmart,Walmart,1101 E Prospect Ave,Ponca City,OK,74601,36.741109,-97.071893
534386,sg:ba81871e7110490287334b713d192cd8,Walmart,Walmart,349 Kinston Hwy,Richlands,NC,28574,34.907981,-77.567338
535876,sg:30071dbc2cc041e283c406064c904995,Walmart,Walmart,677 Timpany Blvd,Gardner,MA,01440,42.551510,-71.994626
537429,sg:ac58140ad6634fc487c93b0f9faf2e49,Walmart,Walmart,8450 La Palma Ave,Buena Park,CA,90620,33.844659,-117.986690


In [9]:
df_walmart_location.shape,df_walmart_location['safegraph_place_id'].nunique()

((4693, 9), 4693)

# Zips in X miles from the store

In [10]:
radius=10
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [11]:
df_poi_locations=df_walmart_location[['safegraph_place_id','latitude','longitude']].drop_duplicates()
df_poi_locations['coor']=df_poi_locations[['latitude','longitude']].values.tolist()
dict_poi_locations=df_poi_locations.set_index("safegraph_place_id").to_dict()['coor']

In [12]:
df_zip_by_poi=pd.DataFrame()
i_counter=0
for poi_id, poi_coor in dict_poi_locations.items():
    list_zips_in_x_miles=[]
    for zip_cd, zip_centroid in zip_centers.items():
        dist=haversine(poi_coor,zip_centroid,unit="mi")
        if dist<=radius:
            list_zips_in_x_miles.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_zips_in_x_miles},index=[poi_id]*len(list_zips_in_x_miles)).reset_index().rename(columns={"index":"poi_id"})
    
    df_zip_by_poi=df_zip_by_poi.append(df)
    i_counter+=1
    if i_counter%100==0:
        print(i_counter,datetime.datetime.now())

100 2020-05-20 18:38:49.537622
200 2020-05-20 18:39:03.738872
300 2020-05-20 18:39:17.888996
400 2020-05-20 18:39:32.066724
500 2020-05-20 18:39:46.240662
600 2020-05-20 18:40:00.410333
700 2020-05-20 18:40:15.108616
800 2020-05-20 18:40:30.465589
900 2020-05-20 18:40:44.933657
1000 2020-05-20 18:40:59.462717
1100 2020-05-20 18:41:13.684834
1200 2020-05-20 18:41:27.895966
1300 2020-05-20 18:41:42.054418
1400 2020-05-20 18:41:56.245453
1500 2020-05-20 18:42:10.514963
1600 2020-05-20 18:42:25.043286
1700 2020-05-20 18:42:39.741030
1800 2020-05-20 18:42:54.091142
1900 2020-05-20 18:43:08.419160
2000 2020-05-20 18:43:22.743157
2100 2020-05-20 18:43:37.044590
2200 2020-05-20 18:43:51.360666
2300 2020-05-20 18:44:05.702853
2400 2020-05-20 18:44:20.009959
2500 2020-05-20 18:44:34.293553
2600 2020-05-20 18:44:48.612737
2700 2020-05-20 18:45:02.927801
2800 2020-05-20 18:45:17.218950
2900 2020-05-20 18:45:31.475387
3000 2020-05-20 18:45:45.746254
3100 2020-05-20 18:46:00.016514
3200 2020-05-20 1

In [13]:
print(df_zip_by_poi.shape)
df_zip_by_poi.head(4)

(97329, 2)


,poi_id,zip_cd
0,sg:43f385d5026e4fffb55bbd42f0a91302,32744
1,sg:43f385d5026e4fffb55bbd42f0a91302,32713
2,sg:43f385d5026e4fffb55bbd42f0a91302,32725
3,sg:43f385d5026e4fffb55bbd42f0a91302,32724


In [14]:
df_unique_zip_cd_in_10_miles=df_zip_by_poi[['zip_cd']].drop_duplicates()
df_unique_zip_cd_in_10_miles.shape

(24126, 1)

# Demo by zip_cd

In [74]:
# demo needed:
'''
population
1. age 25-54 (applied the census with the mapping pctg)
2. parents with children<age 18 (from EASI zip 4, single with child hh + married with child hh *2)
'''  

# Demo data
'''
1. zip-4 level EASI (A,B,C 3 parts): "/home/jian/Docs/EASI/2018-07-19/"
2. ZCTA level Census from the factfinder, need to download new if you want something you need: "/home/jian/Docs/Household_and_Population/Cencus_2019/"
3. ZCTA to zip_cd (postal code in 5 digit) mapping: "/home/jian/Docs/Household_and_Population/Cencus_2019/zcta_zip_mapping/zip_to_zcta_2018.xlsx"
'''

'\n1. zip-4 level EASI (A,B,C 3 parts): "/home/jian/Docs/EASI/2018-07-19/"\n2. ZCTA level Census from the factfinder, need to download new if you want something you need: "/home/jian/Docs/Household_and_Population/Cencus_2019/"\n3. ZCTA to zip_cd (postal code in 5 digit) mapping: "/home/jian/Docs/Household_and_Population/Cencus_2019/zcta_zip_mapping/zip_to_zcta_2018.xlsx"\n'

In [75]:
df_EASI_C=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_C2_CSV/ZIP4_18_DATA_C2.CSV",
                      skiprows=1,usecols=['ZIP_CODE','ZIP4','HH18','PCTHHL25','PCTHH25_34','PCTHH35_44','PCTHH45_54','PCTMARC','PCTSPARC'])
df_EASI_C['ZIP_CODE']=df_EASI_C['ZIP_CODE'].astype(str).apply(lambda x: x.zfill(5))
df_EASI_C['ZIP4']=df_EASI_C['ZIP4'].astype(str).apply(lambda x: x.zfill(4))

df_EASI_C['HH_0_25']=df_EASI_C['HH18']*df_EASI_C['PCTHHL25']/100
df_EASI_C['HH_25_34']=df_EASI_C['HH18']*df_EASI_C['PCTHH25_34']/100
df_EASI_C['HH_35_44']=df_EASI_C['HH18']*df_EASI_C['PCTHH35_44']/100
df_EASI_C['HH_45_54']=df_EASI_C['HH18']*df_EASI_C['PCTHH45_54']/100
df_EASI_C['HH_25_54']=df_EASI_C['HH_25_34']+df_EASI_C['HH_35_44']+df_EASI_C['HH_45_54']

df_EASI_C['HH_MARC']=df_EASI_C['HH18']*df_EASI_C['PCTMARC']/100
df_EASI_C['HH_SPARC']=df_EASI_C['HH18']*df_EASI_C['PCTSPARC']/100
df_EASI_C['HH_With_Children']=df_EASI_C['HH_MARC']+df_EASI_C['HH_SPARC']
df_EASI_C['parents_with_Children']=df_EASI_C['HH_MARC']*2+df_EASI_C['HH_SPARC']

df_zip_HH=df_EASI_C.groupby(["ZIP_CODE"])['HH18','parents_with_Children'].sum().reset_index()
del df_EASI_C
gc.collect()
df_zip_HH.head(2)

,ZIP_CODE,HH18,parents_with_Children
0,00501,0.0,0.0000
1,01001,6853.6,2580.7664


In [76]:
df_zip_HH=df_zip_HH.rename(columns={"HH18":"Households"})
df_zip_HH.head(4)

,ZIP_CODE,Households,parents_with_Children
0,00501,0.0,0.00000
1,01001,6853.6,2580.76640
2,01002,9835.6,3526.27847
3,01003,0.0,0.00000


In [77]:
df_zip_ZCTA_mapping=pd.ExcelFile("/home/jian/Docs/Household_and_Population/Cencus_2019/zcta_zip_mapping/zip_to_zcta_2018.xlsx")
print(df_zip_ZCTA_mapping.sheet_names)
df_zip_ZCTA_mapping=df_zip_ZCTA_mapping.parse("ziptozcta2017",dtype=str)
print(df_zip_ZCTA_mapping.shape)
df_zip_ZCTA_mapping.head(2)

df_zip_ZCTA_mapping=df_zip_ZCTA_mapping[['ZIP_CODE',"ZCTA"]].drop_duplicates()
print("shape",df_zip_ZCTA_mapping.shape)
print("ZIP_CODE nunique",df_zip_ZCTA_mapping['ZIP_CODE'].nunique())
print("ZCTA nunique",df_zip_ZCTA_mapping['ZCTA'].nunique())
df_zip_ZCTA_mapping.head(5)

['ziptozcta2017', 'source']
(41160, 6)
shape (41160, 2)
ZIP_CODE nunique 41160
ZCTA nunique 33138


,ZIP_CODE,ZCTA
0,00501,11742
1,00544,11742
2,00601,00601
3,00602,00602
4,00603,00603


In [78]:
census_data_S0101=pd.read_csv("/home/jian/Docs/Household_and_Population/Cencus_2019/ACS_17_5YR_S0101_with_ann.csv",dtype=str,skiprows=1)
print(census_data_S0101.shape)

census_population_needed=census_data_S0101[['Id2','Total; Estimate; Total population',
                                            'Total; Estimate; AGE - 25 to 29 years',
                                            'Total; Estimate; AGE - 30 to 34 years',
                                            'Total; Estimate; AGE - 35 to 39 years',
                                            'Total; Estimate; AGE - 40 to 44 years',
                                            'Total; Estimate; AGE - 45 to 49 years',
                                            'Total; Estimate; AGE - 50 to 54 years'
                                           ]]

for col in census_population_needed.columns.tolist()[1:]:
    census_population_needed[col]=census_population_needed[col].astype(float)
census_population_needed['population_25_54']=census_population_needed[census_population_needed.columns.tolist()[1:]].sum(axis=1)
census_population_needed=census_population_needed.rename(columns={"Id2":"ZCTA","Total; Estimate; Total population":"total_population"})
census_population_needed=census_population_needed[['ZCTA','total_population','population_25_54']]
census_population_needed.head(3)

(33120, 459)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,ZCTA,total_population,population_25_54
0,00601,17599.0,24263.0
1,00602,39209.0,54729.0
2,00603,50135.0,69221.0


In [79]:
census_population_needed.head(2)

,ZCTA,total_population,population_25_54
0,00601,17599.0,24263.0
1,00602,39209.0,54729.0


In [80]:
df_census=pd.merge(df_zip_ZCTA_mapping,census_population_needed,on="ZCTA",how="outer")
df_census_by_ZCTA=df_census.groupby(['ZCTA'])['total_population'].sum().to_frame().reset_index()
df_census_by_ZCTA=df_census_by_ZCTA.rename(columns={"total_population":"ZCTA_level_pop"})
df_census=pd.merge(df_census,df_census_by_ZCTA,on="ZCTA",how="left")
print(df_census[pd.isnull(df_census['ZIP_CODE'])].shape)
df_census=df_census[pd.notnull(df_census['ZIP_CODE'])]

(6, 5)


In [81]:
print(df_zip_HH.shape,df_zip_HH['ZIP_CODE'].nunique())
print(df_census.shape,df_census['ZIP_CODE'].nunique())

(39602, 3) 39602
(41160, 5) 41160


In [82]:
df_output=pd.merge(df_zip_HH,df_census,on="ZIP_CODE",how="left")
df_output['zip_cd_pctg_of_ZCTA']=df_output['total_population']/df_output['ZCTA_level_pop']
df_output['population_25_54_applied']=df_output['population_25_54']*df_output['zip_cd_pctg_of_ZCTA']
df_output['total_population_applied']=df_output['total_population']*df_output['zip_cd_pctg_of_ZCTA']

In [83]:
df_output=df_output[['ZIP_CODE','Households','parents_with_Children','total_population_applied','population_25_54_applied']]

In [84]:
writer=pd.ExcelWriter("./To_Joann_US_zips_selected_demo_by_zip_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"based_on_EASI_and_census",index=False)
df_unique_zip_cd_in_10_miles.to_excel(writer,"Walmart_10_miles",index=False)
writer.save()